# Introduction

**Contents:**

1. Introduces the `GEModelTools`
1. Solves and simulates a simple **Heterogenous Agent Neo-Classical (HANC) model**

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from HANCModel import HANCModelClass

### New model

1. Households: Solve
\begin{align*}
v_{t}(z_{t},a_{t-1})	& =\max_{c_{t}}\frac{c_{t}^{1-\sigma}}{1-\sigma}+\beta\mathbb{E}_{t}\left[v_{t+1}(z_{t+1},a_{t})\right] \\
\text{s.t. }a_{t}+c_{t}	& =(1+r_{t})a_{t-1}+(1-\tau_{t})z_{t}\geq0 \\
\log z_{t+1}	&=\rho_{z}\log z_{t}+\psi_{t+1}\,\,\,,\psi_{t}\sim\mathcal{N}(\mu_{\psi},\sigma_{\psi}),\,\mathbb{E}[z_{t}]=1
\end{align*}
where $r_{t}$ is the real-interest rate and $\tau_{t}$ is a tax rate
2. Government: Set taxes and government bonds follows 
\begin{align*} 
B_{t+1}=(1+r_{t})B_{t}-\int\tau_{t}z_{t}d\boldsymbol{D}_{t} 
\end{align*}

3. Bond market clearing: $B_{t}=\int a_{t}^{\ast}(z_{t},a_{t-1})d\boldsymbol{D}_{t}$

4. Define and find the stationary equilibrium

\begin{align*}
B_{t+1}=(1+r_{t})B_{t}-\int\tau_{t}z_{t}d\boldsymbol{D}_{t} \\
\Rightarrow B_{ss}=(1+r_{ss})B_{ss}-\int\tau_{t}z_{t}d\boldsymbol{D}_{ss} \\
\Leftrightarrow r_{ss}=\frac{1}{B_{ss}}\int\tau_{t}z_{t}d\boldsymbol{D}_{ss}
\end{align*}

\begin{align*}
H_{ss}\left(B_{ss}; \tau_{ss}\right)=
\left[ 
    \begin{matrix}
B_{ss}-\boldsymbol{a}_{ss}^{\ast\prime}\boldsymbol{D}_{ss} \\
r_{ss}-\frac{1}{B_{ss}}\int\tau_{ss}z_{t}d\boldsymbol{D}_{ss} \\
\boldsymbol{D}_{ss}-\Pi_{z}^{\prime}\underline{\boldsymbol{D}}_{ss} \\
\underline{\boldsymbol{D}}_{ss}-\Lambda_{ss}^{\prime}\boldsymbol{D}_{ss}
\end{matrix} 
\right] =\boldsymbol{0}
\end{align*}
With Policy function: $a_{t}^{\ast}=a^{\ast}\left(\left\{ r_{\tau},\tau_{ss}\right\} _{\tau\geq t}\right)$

And Choice transition: $\Lambda_{t}=\Lambda\left(\left\{ r_{\tau},\tau_{ss}\right\} _{\tau\geq t}\right)$

You can also back out tau:
\begin{align*}
\Leftrightarrow \tau_{ss} = \frac{r_{ss}B_{ss}}{\int z_{t}d\boldsymbol{D}_{ss}}
\end{align*}
But because it part of the household problem we cannot use the indirect solution method 

# Setup

In [2]:
model = HANCModelClass(name='A1') # create an instance of the model

In [3]:
ss =model.ss
par  =model.par
ss.r = 0.02
ss.w = 1.

# Preset calibration
ss.taua = 0.1
ss.taul = 0.3
ss.G = 0.3


**Solve the household problem** with `.solve_hh_ss()`:
    
1. Calls `.prepare_hh_ss()`
1. Calls `.solve_backwards_hh()` until convergence

In [5]:
par = model.par
ss = model.ss

In [40]:
model.solve_hh_ss(do_print=True)

household problem in ss solved in 0.0 secs [0 iterations]


In [41]:
model.simulate_hh_ss(do_print=True)

household problem in ss simulated in 0.0 secs [2 iterations]


**Aggregate savings:**

In [ ]:
(ss.D.T@par.zeta).shape

# Find stationary equilibrium

## Direct approach

In [ ]:
model.find_ss(method='direct',do_print=True)


**Look at the steady state variables:**

In [ ]:
for varname in model.varlist:
    print(f'{varname:15s}: {ss.__dict__[varname]:.4f}')

In [ ]:
ss = model.ss


## Looking at the stationary equilibrium

### Policy functions

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)
par = model.par
I = par.a_grid < 500

# a. consumption
ax = fig.add_subplot(1,2,1)
ax.set_title(f'consumption')

for i_z,z in enumerate(par.z_grid):
    if i_z%3 == 0 or i_z == par.Nz-1:
        ax.plot(par.a_grid[I],ss.c[0,i_z,I],label=f'z = {z:.2f}')

ax.legend(frameon=True)
ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('consumption, $c_t$')

# b. saving
ax = fig.add_subplot(1,2,2)
ax.set_title(f'saving')

for i_z,z in enumerate(par.z_grid):
    if i_z%3 == 0 or i_z == par.Nz-1:
        ax.plot(par.a_grid[I],ss.a[0,i_z,I]-par.a_grid[I],label=f'z = {z:.2f}')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('savings change, $a_{t}-a_{t-1}$')

fig.tight_layout()

### Distributions

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# a. income
ax = fig.add_subplot(1,2,1)
ax.set_title('productivity')
ax.plot(par.z_grid,np.cumsum(np.sum(ss.D,axis=(0,2))))

ax.set_xlabel('productivity, $z_{t}$')
ax.set_ylabel('CDF')

# b. assets
ax = fig.add_subplot(1,2,2)
ax.set_title('savings')
ax.plot(np.insert(par.a_grid,0,par.a_grid[0]),np.insert(np.cumsum(np.sum(ss.D,axis=(0,1))),0,0.0),label='discrete')
ax.set_xlabel('assets, $a_{t}$')
ax.set_ylabel('CDF')
ax.set_xscale('symlog')

**Income moments:**

In [ ]:
mean_z = np.sum(ss.D*par.z_grid[:,np.newaxis])
std_z = np.sqrt(np.sum(ss.D*(par.z_grid[np.newaxis,:,np.newaxis]-mean_z)**2))
print(f'mean z: {mean_z:5.2f}')
print(f'std. z: {std_z:5.2f}')

**Asset moments:**

In [ ]:
# a. prepare
Da = np.sum(ss.D,axis=(0,1))
Da_cs = np.cumsum(Da)
mean_a = np.sum(Da*par.a_grid)
std_a = np.sqrt(np.sum(Da*(par.a_grid-mean_a)**2))

def percentile(par,Da_cs,p):
    
    # a. check first
    if p < Da_cs[0]: return par.a_grid[0]
    
    # b. find with loop
    i = 0
    while True:
        if p > Da_cs[i+1]:
            if i+1 >= par.Na: raise Exception()
            i += 1
            continue
        else:
            w = (p-Da_cs[i])/(Da_cs[i+1]-Da_cs[i])
            diff = par.a_grid[i+1]-par.a_grid[i]
            return par.a_grid[i]+w*diff
        
p25_a = percentile(par,Da_cs,0.25)
p50_a = percentile(par,Da_cs,0.50)
p95_a = percentile(par,Da_cs,0.95)
p99_a = percentile(par,Da_cs,0.99)

# b. print
print(f'mean a: {mean_a:6.3f}')
print(f'p25  a: {p25_a:6.3f}')
print(f'p50  a: {p50_a:6.3f}')
print(f'p95  a: {p95_a:6.3f}')
print(f'p99  a: {p99_a:6.3f}')
print(f'std. a: {std_a:6.3f}')

**MPC:**

In [ ]:
def calc_MPC(par,ss):
    
    MPC = np.zeros(ss.D.shape)
    dc = (ss.c[:,:,1:]-ss.c[:,:,:-1])
    dm = (1+model.ss.r)*par.a_grid[np.newaxis,np.newaxis,1:]-(1+model.ss.r)*par.a_grid[np.newaxis,np.newaxis,:-1]
    MPC[:,:,:-1] = dc/dm
    MPC[:,:,-1] = MPC[:,:,-1] # assuming constant MPC at end
    mean_MPC = np.sum(MPC*ss.D)
    return mean_MPC

mean_MPC = calc_MPC(par,ss)
print(f'mean MPC: {mean_MPC:.3f}')

**Question:** What is the correlation between income and savings?

## Indirect approach

In [ ]:
# Indirect approach does not work
#model_indirect = model.copy()
#model_indirect.r_ss_target = 0.1
#model_indirect.allocate()

In [ ]:
#model_indirect.find_ss(method='indirect',do_print=True)

**Question:** What are the pros and cons of the direct and indirect method?

# Idiosyncratic risk and the steady state interest rate

In [ ]:
print(f'ss.A_hh = ss.K = {ss.A_hh:.2f}')
print(f'ss.r = {ss.r*100:.2f} %')
print('')
      
for sigma_psi in np.linspace(par.sigma_psi,2*par.sigma_psi,5):
    
    print(f'{sigma_psi = :.2f}')

    model_ = model.copy()
    model_.par.sigma_psi = sigma_psi
        
    model_.solve_hh_ss(do_print=False)
    model_.simulate_hh_ss(do_print=False)
    
    A_hh = np.sum(model_.ss.a*model_.ss.D)
    
    print(f'PE {A_hh = :.2f}')
          
    model_.find_ss(method='direct')

    print(f'GE ss.r = {model_.ss.r*100:.2f} %')
    print(f'GE ss.A_hh = ss.K = {model_.ss.A_hh:.2f}')

    print('')

# Calibration

In [ ]:
from root_finding import brentq

In [ ]:
def calib_obj(beta,model):
    """ calibration objective """
    
    model.par.beta = beta
    model.find_ss(method='direct')    
    
    mean_MPC = calc_MPC(model.par,model.ss)
     
    return mean_MPC-0.20

In [ ]:
model_calib = model.copy()
brentq(calib_obj,0.94,par.beta,args=(model_calib,),do_print=True,varname='beta',funcname='MPC-0.20');

In [ ]:
print(f'ss.r = {model_calib.ss.r*100:.2f} %')
print(f'ss.B = {model_calib.ss.B:.2f}')

**Question:** What could be an alternative be to use a root-finder?

# Exercise

1. What is the optimal level of of $\tau_t$

In [ ]:
model.ss.c.shape

In [ ]:
ss = model.ss


ss.c.shape

In [ ]:
ss.D.shape

In [ ]:
def util(c,par):
    return c**(1-par.sigma)/(1-par.sigma)

def wellfare(model,type='benthamite'):
    ss = model.ss
    par = model.par

    U = util(ss.c,par)
    #Da = np.sum(ss.D,axis=(0,1))

    # Social welfare functions: https://en.wikipedia.org/wiki/Social_welfare_function
    if type.lower() in ['benthamite','sum']:
        
        
        W = np.sum(ss.D*U)
    

    if type.lower() in ['semi-rawls']:
        
        pass
        #W = percentile(par,Da_cs,0.1,var='c')

    return W

In [ ]:
wellfare(model)

In [ ]:
def obj_fun(tau,model):
    model.par.tau = tau
    model.find_ss(method='direct')    
    W = wellfare(model)
    return -W

In [ ]:
model.ss.r

In [ ]:
tau_list = np.linspace(0,0.5,10)


In [ ]:
model_calib = model.copy()
W_list = np.empty(tau_list.shape)

for i, tau in enumerate(tau_list):
    print(f'Tau = {tau:.3f}')

    W = obj_fun(tau,model_calib)
    W_list[i] = W

    print(f'W = {W:.4f}')
    print(f'B = {model_calib.ss.B}')
    print('')


#W_list = [obj_fun(tau,model_calib) for tau in tau_list]
#W_list

In [ ]:
model_calib.par.tau = 0.8
model_calib.find_ss(method='direct',do_print=True) 

In [ ]:
model_calib.par.tau = 0.9
model_calib.find_ss(method='direct',do_print=True) 

In [ ]:
res = optimize.minimize_scalar(obj_fun,bounds=[0.05,0.15],args=(model_calib,),method='Bounded')
res

In [ ]:
tau_list = np.linspace(0,0.95,100)
model_calib = model.copy()
W_list = np.empty(tau_list.shape)

for i, tau in enumerate(tau_list):
    W = obj_fun(tau,model_calib)
    W_list[i] = -W


In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# a. income
ax = fig.add_subplot(1,1,1)
ax.set_title('W over tau')
ax.plot(tau_list,W_list)

ax.set_xlabel('tau')
ax.set_ylabel('W');
